__File: test_notebook.ipynb__

__Author:   A. J. Tropiano (tropiano.4@osu.edu)__<br/>
__Date:     Jun 16, 2021__

Jupyter notebook for testing out code or functions.

__Revision history:<br/>__
* Jun 16, 2021 - Comparing integration via looping vs using NumPy sum function.

In [1]:
import numpy as np
from numpy.polynomial.legendre import leggauss
import time
# Scripts made by A.T.
from Misc.integration import gaussian_quadrature_mesh

In [2]:
def test_func_fast(a_array, b_array, c_array, d_array):
    
    # Create 4-D meshes for integration
    a_mesh, b_mesh, c_mesh, d_mesh = np.meshgrid(a_array, b_array, c_array, d_array, indexing='ij')
    
    value = ( a_mesh**2 + b_mesh ) * c_mesh + a_mesh * d_mesh
    
    return value

In [3]:
def test_func_slow(a, b, c, d):
    
    value = ( a**2 + b ) * c + a * d
    
    return value

In [4]:
# Get four meshes
atot = 100
a_array, a_weights = gaussian_quadrature_mesh(100.0, atot)

btot = 50
b_array, b_weights = gaussian_quadrature_mesh(50.0, btot)

ctot = 10
c_array, c_weights = gaussian_quadrature_mesh(1.0, ctot)

dtot = 8
d_array, d_weights = leggauss(dtot)

In [5]:
# Integrate doing the slow way

t0 = time.time()
integrated_value = 0
for ia, a in enumerate(a_array):
    aw = a_weights[ia]
    for ib, b in enumerate(b_array):
        bw = b_weights[ib]
        for ic, c in enumerate(c_array):
            cw = c_weights[ic]
            for i_d, d in enumerate(d_array):
                dw = d_weights[i_d]
                
                integrated_value += test_func_slow(a, b, c, d) * dw * cw * bw * aw


t1 = time.time()
seconds = t1-t0

print(integrated_value)
print('%.5f seconds elapsed.' % seconds)

16791666.666666236
0.53680 seconds elapsed.


In [6]:
# Integrate doing the fast way

# Get weights as 4-D meshes
aw_mesh, bw_mesh, cw_mesh, dw_mesh = np.meshgrid(a_weights, b_weights, c_weights, d_weights, indexing='ij')
weights_mesh = aw_mesh * bw_mesh * cw_mesh * dw_mesh
full_integrand = test_func_fast(a_array, b_array, c_array, d_array) * weights_mesh

t0 = time.time()
integrated_value = np.sum( np.sum( np.sum( np.sum(full_integrand, axis=-1), axis=-1), axis=-1), axis=-1)
t1 = time.time()
seconds = t1-t0

print(integrated_value)
print('%.5f seconds elapsed.' % seconds)

16791666.666666623
0.00138 seconds elapsed.
